## 예제
1. test_csv폴더에 있는 '유통+업종(백화점,+할인점,+슈퍼마켓,+편의점)+카드소비트렌드+데이터_1901_2109'파일을 로드 
2. 각 컬럼의 이름을 ['기준연월', '업종', '성별', '연령대', '총이용건수', '총이용금액']
3. 기준년월을 시계열데이터로 변경
4. 성별 컬럼의 데이터를 1은 남자, 2는 여자로 변경, 9는 무관
5. 남, 녀 성별을 기준으로 연도별 건당 이용금액의 평균이 몇인지 출력

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("../test_csv/유통+업종(백화점,+할인점,+슈퍼마켓,+편의점)+카드소비트렌드+데이터_1901_2109.csv", encoding='euc-kr')

In [ ]:
df.head()

In [ ]:
df.columns = ['기준년월', '업종', '성별', '연령대', '총이용건수', '총이용금액']

In [ ]:
df.head()

In [ ]:
# 판다스에서 지수표현 없애기 
pd.options.display.float_format = '{:2f}'.format

In [ ]:
# 기준년월 데이터를 시계열 변경
df['기준년월'] = pd.to_datetime(df['기준년월'], format='%Y%m')

In [ ]:
# 인덱스를 기준년월로 설정
df.set_index('기준년월', inplace=True)

In [ ]:
# 괴상한 데이터 
df['연령대'].value_counts()

In [ ]:
# 성별 컬럼에서 1은 남자 2는 여자 9는 무관

# case1
def change(x):
    if x == 1:
        result = '남자'
    elif x == 2:
        result = '여자'
    elif x == 9:
        result = '무관'
    else:
        result = "-"
    return result

df['성별'].apply(change).value_counts()

In [ ]:
df['성별'].apply(lambda x : "남자" if (x == 1) else ("여자" if (x ==2) else "무관"))

In [ ]:
df['성별'] = df['성별'].replace(
    {
        1 : '남자', 
        2 : '여자', 
        9 : "무관"
    }
)

In [ ]:
df['성별'].value_counts()

In [ ]:
# 연도 파생변수 생성
df['연도'] = df.index.strftime('%Y')

In [ ]:
df.head()

In [ ]:
# 건당 이용금액 파생변수 생성
# 총이용금액 / 총이용건수
df['건당이용금액'] = df['총이용금액'] / df['총이용건수']

In [ ]:
df.head()

In [ ]:
import platform

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# 그래프에서 한글 깨짐 방지 
if platform.system() == "Darwin":    # MAC 인 경우
    plt.rc('font', family = 'AppleGothic')
else:
    plt.rc('font', family = 'Malgun Gothic')

In [ ]:
(df[['연도', "성별" ,'건당이용금액']].groupby(['연도', '성별']).mean()).plot.bar()

In [ ]:
pivot = pd.pivot_table(
    df, 
    index= '연도', 
    columns = '성별', 
    aggfunc = "mean", 
    values = '건당이용금액'
)

In [ ]:
pivot.plot.bar()

### 온라인 카드 매출과 오프라인 카드 매출을 월별로 비교
1. 오프라인 데이터 df를 월별로 그룹화 총이용금액의 합계
    - 인덱스는 기준년월을 시계열로 변경한 데이터로 설정
2. 온라인 데이터 df_online도 월별로 그룹화 총이용금액의 합계
    - 인덱스는 기준년월을 시계열로 변경한 데이터로 설정

In [ ]:
df.head()

In [ ]:
group_data = df.groupby(df.index).sum()['총이용금액']

In [ ]:
df_online = pd.read_csv("../test_csv/온라인+카드소비트렌드+데이터_1901_2109.csv")

In [ ]:
df_online.head()

In [ ]:
# 컬럼의 이름을 변경
df_online.columns = ['기준년월', '성별', '연령대', '총이용건수', '총이용금액']

In [ ]:
# 기준년월을 시계열로 변경하고 인덱스로 설정
df_online['기준년월'] = pd.to_datetime(df_online['기준년월'], format='%Y%m')

In [ ]:
df_online.set_index('기준년월', inplace=True)

In [ ]:
# 그룹화
group_data2 = df_online.groupby('기준년월').sum()['총이용금액']

In [ ]:
# 그래프로 시각화

# 그래프의 사이즈 변경
plt.figure(figsize = (14, 6))

# grid 추가
plt.grid(True, axis = 'y')

# 지수표현 제거 
plt.ticklabel_format(axis = 'y', style='plain')

plt.plot(group_data, label='오프라인')
plt.plot(group_data2, label = '온라인')
plt.title('온라인/오프라인 카드매출 비교')
plt.legend()
plt.show()

In [ ]:
df

In [ ]:
df_online['업종'] = "온라인"

In [ ]:
df_online.head()

In [ ]:
total_df = pd.concat([df, df_online], axis='rows')

In [85]:
total_df['업종'].value_counts()

편의점     667
백화점     664
슈퍼마켓    663
할인점     654
온라인     651
Name: 업종, dtype: int64